# 类工厂函数

In [2]:
# 简单版本
from typing import Any
from collections.abc import Iterable, Iterator

FieldNames = str | Iterable[str]

def record_factory(cls_name: str, field_names: FieldNames) -> type[tuple]:

    slots = parse_identifiers(field_names)

    def __init__(self, *args, **kwargs) -> None:
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self) -> Iterator[Any]:
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):
        values = ', '.join(f'{name}={value!r}' for name, value in zip(self.__slots__, self))
        cls_name = self.__class__.__name__
        return f'{cls_name}({values})'

    cls_attrs = dict(__slots__=slots,  # 类属性字典
                     __init__=__init__,
                     __iter__=__iter__,
                     __repr__=__repr__,)
    return type(cls_name, (object,), cls_attrs)  # 构建新类

def parse_identifiers(names: FieldNames) -> tuple[str, ...]:
    if isinstance(names, str):
        names = names.replace(',', ' ').split()
    if not all(s.isidentifier() for s in names):  # 如果存在非Python合法字符名
        raise ValueError('names must all be valid identifiers')
    return tuple(names)

In [4]:
Dog = record_factory('Dog', 'name weight owner')
rex = Dog('Rex', 30, 'Bob')
rex

Dog(name='Rex', weight=30, owner='Bob')

In [6]:
name, weight, _ = rex
name, weight

('Rex', 30)

In [7]:
rex.weight = 32
rex

Dog(name='Rex', weight=32, owner='Bob')

In [10]:
Dog.__mro__

(__main__.Dog, object)

# __init_subclass__

In [11]:
from collections.abc import Callable
from typing import Any, NoReturn, get_type_hints


In [12]:
class Field:
    def __init__(self, name: str, constructor: Callable) -> None:
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        self.constructor = constructor

    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...:
            value = self.constructor()
        else:
            try:
                value = self.constructor(value)
            except (TypeError, ValueError) as e:
                type_name = self.constructor.__name__
                msg = f'{value!r} is not compatible with {self.name}:{type_name}'
                raise TypeError(msg) from e
        instance.__dict__[self.name] = value

In [13]:
from typing import Any


class Checked:
    @classmethod
    def _fields(cls) -> dict[str, type]:
        return get_type_hints(cls)

    def __init_subclass__(subclass) -> None:  # 非常规类方法
        super().__init_subclass__()
        for name, constructor in subclass._fields().items():
            setattr(subclass, name, Field(name, constructor))

    def __init__(self, **kwargs: Any) -> None:
        for name in self._fields():
            value = kwargs.pop(name, ...)
            setattr(self, name, value)
        if kwargs:
            self.__flag_unknown_attrs(*kwargs)

    def __setattr__(self, __name: str, __value: Any) -> None:
        if __name in self._fields():
            cls = self.__class__

            descriptor = getattr(cls, __name)
            descriptor.__set__(self, __value)
        else:
            self.__flag_unknown_attrs(name)

    def __flag_unknown_attrs(self, *names: str) -> NoReturn:
        plural = 's' if len(names) > 1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')

    def _asdict(self) -> dict[str, Any]:
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items()
            if isinstance(attr, Field)
        }

    def __repr__(self) -> str:
        kwargs = ', '.join(
            f'{key}={value!r}' for key, value in self._asdict().items()
        )
        return f'{self.__class__.__name__}({kwargs})'

In [14]:
class Movie(Checked):
    title: str
    year: int
    box_office: float

movie = Movie(title='The Godfather', year=1972, box_office=137)
movie.title

'The Godfather'

In [15]:
movie

Movie(title='The Godfather', year=1972, box_office=137.0)

# 另一种实现，通过类装饰器

In [17]:
def _fields(cls: type) -> dict[str, type]:
    return get_type_hints(cls)

def __init__(self, **kwargs: Any) -> None:
    for name in self._fields():
        value = kwargs.pop(name, ...)
        setattr(self, name, value)
    if kwargs:
        self.__flag_unknown_attrs(*kwargs)

def __setattr__(self, __name: str, __value: Any) -> None:
    if __name in self._fields():
        cls = self.__class__

        descriptor = getattr(cls, __name)
        descriptor.__set__(self, __value)
    else:
        self.__flag_unknown_attrs(name)

def __flag_unknown_attrs(self, *names: str) -> NoReturn:
    plural = 's' if len(names) > 1 else ''
    extra = ', '.join(f'{name!r}' for name in names)
    cls_name = repr(self.__class__.__name__)
    raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')

def _asdict(self) -> dict[str, Any]:
    return {
        name: getattr(self, name)
        for name, attr in self.__class__.__dict__.items()
        if isinstance(attr, Field)
    }

def __repr__(self) -> str:
    kwargs = ', '.join(
        f'{key}={value!r}' for key, value in self._asdict().items()
    )
    return f'{self.__class__.__name__}({kwargs})'

In [18]:
def checked(cls: type) -> type:
    for name, constructor in _fields(cls).items():
        setattr(cls, name, Field(name, constructor))

    cls._fields = classmethod(_fields)  # type: ignore

    instance_methods = (
        __init__,
        __repr__,
        __setattr__,
        _asdict,
        __flag_unknown_attrs,
    )
    for method in instance_methods:
        setattr(cls, method.__name__, method)

    return cls

In [19]:
@checked
class Movie():
    title: str
    year: int
    box_office: float

movie = Movie(title='The Godfather', year=1972, box_office=137)
movie.title

'The Godfather'

In [20]:
str.__class__

type

In [21]:
type.__class__

type

In [22]:
from collections.abc import Iterable

Iterable.__class__

abc.ABCMeta

In [23]:
import abc
abc.ABCMeta.__class__

type

In [ ]:
class MetaBunch(type):  # 创建新元类
    def __new__(meta_cls, cls_name, bases, cls_dict):

        defaults = {}

        def __init__(self, **kwargs):
            for name, default in defaults.items():
                setattr(self, name, kwargs.pop(name, default))
            if kwargs:
                extra = ', '.join(kwargs)
                raise AttributeError(f'No slots left for: {extra!r}')

        def __repr__(self):
            rep = ', '.join(f'{name}={value!r}'
                            for name, default in defaults.items()
                            if (value := getattr(self, name)) != default)
            return f'{cls_name}({rep})'

        new_dict = dict(__slots__=[],
                        __init__=__init__,
                        __repr__=__repr__)

        for name, value in cls_dict.items():
            if name.startswith('__') and name.endswith('__'):
                if name in new_dict:
                    raise AttributeError(f"Can't set {name!r} in {cls_name!r}")
                new_dict[name] = value
            else:
                new_dict['__slots__'].append(name)
                defaults[name] = value
        return super().__new__(meta_cls, cls_name, bases, new_dict)

class Bunch(metaclass=MetaBunch):
    pass

In [ ]:
class Point()